In [1]:
using AstrobeeRobot
using RigidBodySim
using RigidBodyDynamics

using MeshCat
using MeshCatMechanisms

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/2f9e3fa36abf973ebb00b0114f6fd044f48470cb-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/2f9e3fa36abf973ebb00b0114f6fd044f48470cb-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


### Explanation on Motion

To create an animation of a robot, use the setanimation!(MechanismVisualizer, t, q) command. t represents the timesteps, and q represents the configurations. Because we created a floating joint for the Astrobee, the configuration actually rotates and translates the robot. The configuration is represented as follows: [q1, q2, q3, q4, x, y, z] where q represents the quaternion (with [1,0,0,0] being no rotation) and [x,y,z] is the translation in the respective axis

To set the robot configuration, use the set_configuration!(state, joint, config)  [note: removing the joint section will just default to rotating/translating the robot] command. At the time of writing, it is unknown (and probably unlikely) that set_configuration! can be used by the animation :\ The Astrobee has 6 joints 




In [21]:
vis = Visualizer()
delete!(vis)
ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);
open(vis)
sleep(1)

q = [
    [1;0;0;0;0;0;0],
    [1;0;0;0;1;1;0],
    [0.5;0.5;0.5;0.5;2;2;0] ]

setanimation!(mvis,1:length(q),q)

ab.mechanism

instantiated a floating joint
Listening on 0.0.0.0:8715...


Info: Serving MeshCat visualizer at http://127.0.0.1:8715


Spanning tree:
Vertex: world (root)
  Vertex: body, Edge: body_to_world
    Vertex: inertial_link, Edge: inertial_joint
    Vertex: top_aft, Edge: top_aft
      Vertex: top_aft_arm_proximal_link, Edge: top_aft_arm_proximal_joint
        Vertex: top_aft_arm_distal_link, Edge: top_aft_arm_distal_joint
          Vertex: top_aft_gripper_left_proximal_link, Edge: top_aft_gripper_left_proximal_joint
            Vertex: top_aft_gripper_left_distal_link, Edge: top_aft_gripper_left_distal_joint
          Vertex: top_aft_gripper_right_proximal_link, Edge: top_aft_gripper_right_proximal_joint
            Vertex: top_aft_gripper_right_distal_link, Edge: top_aft_gripper_right_distal_joint
No non-tree joints.

Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")

In [22]:
### Examples of Setting a Configuration

state = MechanismState(ab.mechanism)
#Translate and Rotate the Astrobee State
set_configuration!(state, ab.basejoint, [1; 0; 0; 0; 0; 0; 1.025])

#Operate the Astrobee's arm joints of the State
set_configuration!(state, findjoint(ab.mechanism,"top_aft_arm_proximal_joint"),[1.0])
set_configuration!(state, findjoint(ab.mechanism,"top_aft_arm_distal_joint"),[1.0])
set_configuration!(state, findjoint(ab.mechanism,"top_aft_gripper_left_proximal_joint"),[1.0])
set_configuration!(state, findjoint(ab.mechanism,"top_aft_gripper_left_distal_joint"),[1.0])
set_configuration!(state, findjoint(ab.mechanism,"top_aft_gripper_right_proximal_joint"),[1.0])
set_configuration!(state, findjoint(ab.mechanism,"top_aft_gripper_right_distal_joint"),[1.0])

#Set the Astrobee to be at the state we just defined
set_configuration!(mvis, configuration(state))


    


Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")

In [104]:
using PandaRobot
vis = Visualizer()

pan = Panda()
mvis = MechanismVisualizer(
    pan.mechanism,
    URDFVisuals(PandaRobot.urdfpath(), package_path=[dirname(dirname(PandaRobot.urdfpath()))]),
    vis);

open(vis)
sleep(1)

q = [ 
    [0;0;0;0;0;0;0;0;0],
    [1;1;1;1;1;1;1;0.05;0.05],
    [0;0.5;0.7;0.5;0.3;0.8;0.2;0.02;0.02]
    ]


setanimation!(mvis, 1:length(q), q)
#pan.mechanism

Listening on 0.0.0.0:8790...


Info: Serving MeshCat visualizer at http://127.0.0.1:8790


true

The following code iterates through all of the links to add points based on each link's local frame. 

- this_link_frame is the frame of the local link. This is obtained by the function default_frame(RigidBody)
- this_link_point creates a point centered at the origin of the local frame. The 3 numbers afterwards translate the point in the x, y, and z direction of the local frame
- this_link_radius represents the radius size


For visualization purposes, the "function setelement!(MechanismVisualizer,Point3D, int radius, string name)" can be used. Note, renaming a point the same name as a previously used point will replace the old point

Joints can be found by the function: "findjoint(mechanism, string link_name_from_urdf)". The frames of the joint can be recovered by "frame_before(joint)" or "frame_after(joint)"

To find a body or joint in a mechanism, type:

RigidBodyDynamics.findbody(mechanism, "urdf_link_name")
RigidBodyDynamics.findjoint(mechanism, "urdf_joint_name")

In [120]:
#Interestingly, RigidBodyDynamics features both kinematics and dynamics library; you can call mass matrix from it
#Implies Bullet should have similar functionality; an issue might be translating the information from one to the other
#Look into this

#setelement!(mvis, Point3D(frame_after(findjoint(pan.mechanism,"panda_link0_to_world")),0.0,0.0,0.0), radius, panda_link0_name)

#http://www.juliarobotics.org/RigidBodyDynamics.jl/stable/spatial.html#RigidBodyDynamics.Spatial.Point3D

link_frame = CartesianFrame3D[]
point = Point3D[]
radius = Float64[]
i = 1;
for link in RigidBodyDynamics.bodies(pan.mechanism)
    this_link_frame = RigidBodyDynamics.default_frame(link)
    this_link_point = RigidBodyDynamics.Point3D(this_link_frame,0.0,0.0,0.1)
    this_link_radius = 0.1
    
    push!(link_frame, this_link_frame)
    push!(point,this_link_point)
    push!(radius,this_link_radius)
    
#   for visualization
    MeshCatMechanisms.setelement!(mvis, this_link_point, this_link_radius, string("point_",i))
    i=i+1

end

RigidBodyDynamics.bodies(pan.mechanism)

13-element Array{RigidBodyDynamics.RigidBody{Float64},1}:
 world            
 panda_link0      
 panda_link1      
 panda_link2      
 panda_link3      
 panda_link4      
 panda_link5      
 panda_link6      
 panda_link7      
 panda_link8      
 panda_hand       
 panda_leftfinger 
 panda_rightfinger

A **path** is a graph that shows the coordinate transformations from a joint to another joint. Essentially, it shows how joints are connected to each other.

'path = RigidBodyDynamics.path(mechanism, first_body, last_body)'

Frames can be obtained by
'frame = RigidBodyDynamics.transform(state, frame_from, frame_to)'

**Double check if frame_from /frame_to are in the right order**
state (the mechanism state) provides a way for RigidBodyDynamics to transform the coordinate frame along the known tranformation frames in the manipulator chain

Using a **Point**:
p.frame = local frame of the point
p.v = the vector of the point (x,y,z)




In [126]:
# Jacobian_IK_and_Control notebook in RigidBodyDynamics is a good example 

#bodies(mechanism) returns the links in the chain
body = RigidBodyDynamics.bodies(pan.mechanism)[6]
#root_frame is the world frame of the entire mechanism
world = RigidBodyDynamics.root_frame(pan.mechanism)

#A graph of your joints that you care about, from one joint to another joint. E.g. Joint 1 -> Joint 6
p = RigidBodyDynamics.path(pan.mechanism, root_body(pan.mechanism),body)

#How to create a Mechanism State and enact it upon the robot
state = RigidBodyDynamics.MechanismState(pan.mechanism)
set_configuration!(state,[0;0.4;0.8;0.3;0.5;1.0;0.4;0.02;0.02])
set_configuration!(mvis, configuration(state))

#Create a point in the world frame
desired_point = Point3D(world,0.5,0.0,2.0)

#Get the jacobian for the point given the joints we care about, the state it's in, and the point
Jp = point_jacobian(state,p, transform(state,desired_point,world))
Jp = point_jacobian(state,p,transform(state,point[6],point[6].frame))


#Not sure how transform(configuration, point, frame) is transforming the point. Is it to the end-effector's frame, or the local frame?
#point_jacobian -> if we give an entire manipulator configuration, will this return based on our shorter chain path (probably does)




CartesianFrame3D: "after_panda_joint4" (id = 271)